In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv(r"C:\Users\bhavi\Desktop\House_prize_prediction\Mumbai House Prices.csv")

In [5]:
df.head()

,bhk,type,locality,area,price,price_unit,region,status,age
0,3,Apartment,Lak And Hanware The Residency Tower,685,2.50,Cr,Andheri West,Ready to move,New
1,2,Apartment,Radheya Sai Enclave Building No 2,640,52.51,L,Naigaon East,Under Construction,New
2,2,Apartment,Romell Serene,610,1.73,Cr,Borivali West,Under Construction,New
3,2,Apartment,Soundlines Codename Urban Rainforest,876,59.98,L,Panvel,Under Construction,New
4,2,Apartment,Origin Oriana,659,94.11,L,Mira Road East,Under Construction,New


In [7]:
df.info

<bound method DataFrame.info of        bhk       type                              locality  area  price  \
0        3  Apartment   Lak And Hanware The Residency Tower   685   2.50   
1        2  Apartment     Radheya Sai Enclave Building No 2   640  52.51   
2        2  Apartment                         Romell Serene   610   1.73   
3        2  Apartment  Soundlines Codename Urban Rainforest   876  59.98   
4        2  Apartment                         Origin Oriana   659  94.11   
...    ...        ...                                   ...   ...    ...   
76033    3  Apartment                     Parinee Liva Roca  1527   7.00   
76034    5  Apartment                     Parinee Liva Roca  3049  12.00   
76035    4  Apartment                         Lodha Seaview  3313  10.00   
76036    2  Apartment                        Hubtown Serene  1305   4.25   
76037    5  Apartment              Sunteck Signature Island  5200  25.00   

      price_unit                region              sta

In [9]:
def convert_price(row):
    if row['price_unit'] == 'Cr':
        return float(row['price']) * 100  # 1 Cr = 100 Lakhs
    elif row['price_unit'] == 'L':
        return float(row['price'])
    return np.nan

df['price_in_lakh'] = df.apply(convert_price, axis=1)


In [11]:
df.drop(['price', 'price_unit', 'locality'], axis=1, inplace=True)

In [13]:
import pandas as pd
import numpy as np

# Step 1: Get the indices where age is 'Unknown'
unknown_indices = df[df['age'] == 'Unknown'].index

# Step 2: Shuffle the indices randomly
np.random.seed(42)  # For reproducibility
shuffled_indices = np.random.permutation(unknown_indices)

# Step 3: Split the indices into two halves
half = len(shuffled_indices) // 2
new_indices = shuffled_indices[:half]
resale_indices = shuffled_indices[half:]

# Step 4: Assign "New" to first half, "Resale" to second half
df.loc[new_indices, 'age'] = 'New'
df.loc[resale_indices, 'age'] = 'Resale'


In [17]:
df['price_per_sqft'] = (df['price_in_lakh'] * 100000) / df['area']


In [19]:
df.head(5)

,bhk,type,area,region,status,age,price_in_lakh,price_per_sqft
0,3,Apartment,685,Andheri West,Ready to move,New,250.00,36496.350365
1,2,Apartment,640,Naigaon East,Under Construction,New,52.51,8204.687500
2,2,Apartment,610,Borivali West,Under Construction,New,173.00,28360.655738
3,2,Apartment,876,Panvel,Under Construction,New,59.98,6847.031963
4,2,Apartment,659,Mira Road East,Under Construction,New,94.11,14280.728376


In [53]:
def filter_by_domain(df):
    # Initialize a list to store all filtering conditions
    conditions = []

    # Define domain-specific area and price ranges for each BHK value (in lakhs and sq. ft.)
    bhk_ranges = {
        1: {'area': (300, 600), 'price': (40, 150)},       # 1 BHK: 300-600 sqft, ₹40L–₹150L
        2: {'area': (500, 900), 'price': (60, 250)},       # 2 BHK: 500-900 sqft, ₹60L–₹250L
        3: {'area': (800, 1500), 'price': (100, 400)},     # 3 BHK: 800-1500 sqft, ₹1Cr–₹4Cr
        4: {'area': (1200, 2000), 'price': (200, 600)},    # 4 BHK: 1200-2000 sqft, ₹2Cr–₹6Cr
        5: {'area': (1600, 3000), 'price': (300, 800)},    # 5 BHK: 1600-3000 sqft, ₹3Cr–₹8Cr
        6: {'area': (2000, 3500), 'price': (400, 1000)},   # ...
        7: {'area': (2200, 4000), 'price': (500, 1200)},
        8: {'area': (2500, 4500), 'price': (600, 1500)},
        9: {'area': (3000, 5000), 'price': (700, 1800)},
        10: {'area': (3500, 6000), 'price': (800, 2000)}
    }

    # Loop through each BHK category and define a filter condition based on area and price range
    for bhk, limits in bhk_ranges.items():
        area_min, area_max = limits['area']
        price_min, price_max = limits['price']

        # Create condition for current BHK: area and price must fall within domain-defined range
        condition = (
            (df['bhk'] == bhk) &
            (df['area'] >= area_min) & (df['area'] <= area_max) &
            (df['price_lakh'] >= price_min) & (df['price_lakh'] <= price_max)
        )

        # Add the condition to the list
        conditions.append(condition)

    # Combine all conditions using OR (|) to create a final mask
    final_condition = conditions[0]
    for c in conditions[1:]:
        final_condition |= c

    # Return the filtered DataFrame where at least one condition is satisfied
    return df[final_condition]


In [57]:
df[df['bhk'] == 1].min()


bhk                           1
type                  Apartment
area                        300
region                   Airoli
status            Ready to move
age                         New
price_in_lakh              40.0
price_per_sqft      6666.666667
dtype: object

In [59]:
df.describe()

,bhk,area,price_in_lakh,price_per_sqft
count,27991.000000,27991.000000,27991.000000,27991.000000
mean,2.045372,816.683791,149.770127,18246.148203
std,0.857274,369.531173,90.923952,6416.998855
min,1.000000,300.000000,40.000000,6666.666667
25%,1.000000,550.000000,84.000000,13207.547170
50%,2.000000,725.000000,126.000000,17677.419355
75%,3.000000,1000.000000,190.000000,22727.272727
max,8.000000,3700.000000,985.000000,50000.000000


In [65]:
df.to_csv("cleaned_data.csv")

Training of model

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np

In [72]:
# Load your cleaned data
df = pd.read_csv("cleaned_data.csv")

In [74]:
# Feature and target
X = df[['bhk', 'area']]  # Add categorical encodings if needed
y = df['price_in_lakh']

In [76]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [78]:
# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [79]:
# Predict and evaluate
y_pred = model.predict(X_test)
print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R² Score: 0.6819565111026569
RMSE: 50.75872469256728


In [84]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Assuming df is your cleaned DataFrame with these columns
region_encoder = LabelEncoder()
df['region'] = region_encoder.fit_transform(df['region'])

status_encoder = LabelEncoder()
df['status'] = status_encoder.fit_transform(df['status'])

age_encoder = LabelEncoder()
df['age'] = age_encoder.fit_transform(df['age'])


In [86]:
# Train your model
from sklearn.ensemble import RandomForestRegressor

X = df[['bhk', 'area', 'region', 'status', 'age']]
y = df['price_in_lakh']

model = RandomForestRegressor()
model.fit(X, y)

# Save model and encoders
joblib.dump(model, 'model.pkl')
joblib.dump(region_encoder, 'region_encoder.pkl')
joblib.dump(status_encoder, 'status_encoder.pkl')
joblib.dump(age_encoder, 'age_encoder.pkl')


['age_encoder.pkl']

In [88]:
model = joblib.load('model.pkl')
region_encoder = joblib.load('region_encoder.pkl')
status_encoder = joblib.load('status_encoder.pkl')
age_encoder = joblib.load('age_encoder.pkl')
